<a href="https://colab.research.google.com/github/FatLads/Notebooks/blob/main/DQN_FlatLand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flatland-rl keras-rl2

     |████████████████████████████████| 3.3MB 4.6MB/s 
     |████████████████████████████████| 61kB 5.6MB/s 
  Using cached https://files.pythonhosted.org/packages/ec/7e/4609fd0386d41f0b94fe952708970fb87cc1fb66e088758b1f0ab336802e/tox-3.23.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/70/c7/e8cb4a537ee4fc497ac80a606a667fd1832f28ad3ddbfa25bf30473eae13/pytest-4.6.11-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/40/96/9024a1c07bbe5e16bdcbcbd021b608e37b32df4301ae2090aad27c24ffe6/pytest_runner-5.3.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0c/ee/55912b05af8994a190280e3281a18720f8d69da02dcb7ff44e1b96974345/crowdai_api-0.1.22.tar.gz
  Using cached https://files.pythonhosted.org/packages/60/9a/835ba329e31aa471a5597c733f7ca0136b3a0622ce01b9e66b40f5909da4/recordtype-1.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a8/7b/630049fc4af9e68a625738612edc264ce7cb586c5001a2d4d2209a

In [2]:
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_generators import sparse_rail_generator
from flatland.envs.schedule_generators import sparse_schedule_generator
from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

In [19]:
class RailProcessor(Processor):
    def process_observation(self, observation):

        return np.array(observation[0][0][0]).flatten() # For now, we'll just keep the transition maps
    def process_reward(self, reward):
        reward_sum = 0
        for _, train_reward in reward:
            reward_sum += train_reward
        return reward_sum

In [4]:
from flatland.utils.rendertools import RenderTool
import matplotlib.pyplot as plt

def render_env(env, figsize=(8, 8)):
  """Show the environment using matplotlib"""
  env_renderer = RenderTool(env, gl="PILSVG")
  # img is a numpy array
  img = env_renderer.render_env(show=True, return_image=True)

  plt.figure(figsize=figsize)
  plt.imshow(img)
  plt.show()

In [5]:
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_generators import complex_rail_generator
from flatland.envs.observations import GlobalObsForRailEnv

seed = 69

width = 10 # @param{type: "integer"}
height = 10 # @param{type: "integer"}
agents =  2 # @param{type: "integer"}

random_rail_generator = complex_rail_generator(
    nr_start_goal=10, # @param{type:"integer"} number of start and end goals 
                      # connections, the higher the easier it should be for 
                      # the trains
    nr_extra=10, # @param{type:"integer"} extra connections 
                 # (useful for alternite paths), the higher the easier
    min_dist=10,
    max_dist=99999,
    seed=seed
)

env = RailEnv(
    width=width,
    height=height,
    rail_generator=random_rail_generator,
    obs_builder_object=GlobalObsForRailEnv(),
    number_of_agents=agents
)

# env.reset is needed to build the first step of the env
_ = env.reset() # assigned to _ just to suppress the output

In [7]:
nb_actions = env.action_space[0]
nb_actions

5

In [18]:
input_shape = np.array(env.reset()[0][0][0]).flatten().shape
input_shape

(1600,)

In [20]:
model = Sequential()
model.add(Dense(100, input_shape=input_shape, activation="relu"))
model.add(Dense(50, input_shape=input_shape, activation="relu"))
model.add(Dense(100, input_shape=input_shape, activation="relu"))
model.add(Dense(nb_actions, activation="linear"))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 100)               160100    
_________________________________________________________________
dense_9 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_10 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 505       
Total params: 170,755
Trainable params: 170,755
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
memory = SequentialMemory(limit=1000000, window_length=4)
processor = RailProcessor()
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
                              nb_steps=1000000)
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, target_model_update=10000,
               train_interval=4, delta_clip=1.)
dqn.compile(Adam(lr=.00025), metrics=['mae'])

In [24]:
# Okay, now it's time to learn something! We capture the interrupt exception so that training
# can be prematurely aborted. Notice that now you can use the built-in tensorflow.keras callbacks!
weights_filename = 'dqn_flatland_weights.h5f'
checkpoint_weights_filename = 'dqn_flatland_weights_{step}.h5f'
log_filename = 'dqn_flatland_log.json'
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]
dqn.fit(env, callbacks=callbacks, nb_steps=1750000, log_interval=10000)

# After training is done, we save the final weights one more time.
dqn.save_weights(weights_filename, overwrite=True)

# Finally, evaluate our algorithm for 10 episodes.
dqn.test(env, nb_episodes=10, visualize=False)

Training for 1750000 steps ...
Interval 1 (0 steps performed)


ValueError: ignored